# Twitter Sentiment Analysis Notebook

This notebook applies VADER and a Transformer-based model to perform sentiment analysis on tweets.

## 1. Install and Import Libraries
```bash
pip install pandas nltk vaderSentiment transformers torch
```

In [2]:
pip install pandas nltk vaderSentiment transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline

# Download NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## 2. Load the Tweets Dataset
Load the CSV file containing tweets.

In [5]:
file_path = '/content/tweets-data.csv'
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweets,hashtag
0,0,2023-06-25 19:16:20+00:00,0,NaN,@jacksonhinklle #wagner with 6.2 billion dolla...,wagner
1,1,2023-06-25 19:16:18+00:00,0,NaN,Pobrecito es discapacitado\n#Reddetuiterosdemo...,wagner
2,2,2023-06-25 19:16:07+00:00,0,NaN,News from the EIR Daily Alert\n\n“#Putin Addre...,wagner
3,3,2023-06-25 19:15:56+00:00,0,NaN,It's Messi day #Messi𓃵 #Messi36 #Russia #bigst...,wagner
4,4,2023-06-25 19:15:54+00:00,0,NaN,Il passaggio chiave di Machiavelli era questo ...,wagner


## 3. Text Cleaning Function
Define a function to clean tweet text: remove URLs, mentions, hashtags, punctuation, lowercase, remove stopwords, and lemmatize.

In [8]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text)
    text = re.sub(r'http\S+|www\.\S+', '', text)        # remove URLs
    text = re.sub(r'@\w+|#', '', text)                  # remove mentions & hashtags
    text = re.sub(r"[^a-zA-Z ]", "", text.lower())      # keep letters only
    tokens = [
        lemmatizer.lemmatize(w)
        for w in text.split()
        if w not in stop_words and len(w) > 1
    ]
    return " ".join(tokens)

# Apply cleaning to the 'Tweets' column
df['cleaned_text'] = df['Tweets'].apply(clean_text)
df.head()

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweets,hashtag,cleaned_text
0,0,2023-06-25 19:16:20+00:00,0,NaN,@jacksonhinklle #wagner with 6.2 billion dolla...,wagner,wagner billion dollar
1,1,2023-06-25 19:16:18+00:00,0,NaN,Pobrecito es discapacitado\n#Reddetuiterosdemo...,wagner,pobrecito e discapacitadoreddetuiterosdemocrat...
2,2,2023-06-25 19:16:07+00:00,0,NaN,News from the EIR Daily Alert\n\n“#Putin Addre...,wagner,news eir daily alertputin addressed russian pe...
3,3,2023-06-25 19:15:56+00:00,0,NaN,It's Messi day #Messi𓃵 #Messi36 #Russia #bigst...,wagner,messi day messi messi russia bigstage wagner s...
4,4,2023-06-25 19:15:54+00:00,0,NaN,Il passaggio chiave di Machiavelli era questo ...,wagner,il passaggio chiave di machiavelli era questo ...


## 4. VADER Sentiment Analysis
Create a function to compute VADER sentiment label and score, then apply it to add two new columns.

In [9]:
analyzer = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    scores = analyzer.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        label = 'positive'
    elif compound <= -0.05:
        label = 'negative'
    else:
        label = 'neutral'
    return pd.Series([label, compound])

# Apply VADER
vader_results = df['cleaned_text'].apply(vader_sentiment)
vader_results.columns = ['vader_label', 'vader_score']
df = pd.concat([df, vader_results], axis=1)
df.head()

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweets,hashtag,cleaned_text,vader_label,vader_score
0,0,2023-06-25 19:16:20+00:00,0,NaN,@jacksonhinklle #wagner with 6.2 billion dolla...,wagner,wagner billion dollar,neutral,0.00
1,1,2023-06-25 19:16:18+00:00,0,NaN,Pobrecito es discapacitado\n#Reddetuiterosdemo...,wagner,pobrecito e discapacitadoreddetuiterosdemocrat...,positive,0.34
2,2,2023-06-25 19:16:07+00:00,0,NaN,News from the EIR Daily Alert\n\n“#Putin Addre...,wagner,news eir daily alertputin addressed russian pe...,neutral,0.00
3,3,2023-06-25 19:15:56+00:00,0,NaN,It's Messi day #Messi𓃵 #Messi36 #Russia #bigst...,wagner,messi day messi messi russia bigstage wagner s...,neutral,0.00
4,4,2023-06-25 19:15:54+00:00,0,NaN,Il passaggio chiave di Machiavelli era questo ...,wagner,il passaggio chiave di machiavelli era questo ...,neutral,0.00


## 5. Transformer-based Sentiment Analysis
Use a pre-trained transformer model for sentiment analysis (via HuggingFace pipeline).

In [11]:
sentiment_pipeline = pipeline('sentiment-analysis')

# Function to get transformer sentiment
def transformer_sentiment(text):
    result = sentiment_pipeline(text[:512])[0]
    return pd.Series([result['label'].lower(), result['score']])

# Apply transformer model
tf_results = df['cleaned_text'].apply(transformer_sentiment)
tf_results.columns = ['tf_label', 'tf_score']
df = pd.concat([df, tf_results], axis=1)
df.head()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweets,hashtag,cleaned_text,vader_label,vader_score,tf_label,tf_score
0,0,2023-06-25 19:16:20+00:00,0,NaN,@jacksonhinklle #wagner with 6.2 billion dolla...,wagner,wagner billion dollar,neutral,0.00,positive,0.995261
1,1,2023-06-25 19:16:18+00:00,0,NaN,Pobrecito es discapacitado\n#Reddetuiterosdemo...,wagner,pobrecito e discapacitadoreddetuiterosdemocrat...,positive,0.34,negative,0.501147
2,2,2023-06-25 19:16:07+00:00,0,NaN,News from the EIR Daily Alert\n\n“#Putin Addre...,wagner,news eir daily alertputin addressed russian pe...,neutral,0.00,negative,0.938790
3,3,2023-06-25 19:15:56+00:00,0,NaN,It's Messi day #Messi𓃵 #Messi36 #Russia #bigst...,wagner,messi day messi messi russia bigstage wagner s...,neutral,0.00,positive,0.971753
4,4,2023-06-25 19:15:54+00:00,0,NaN,Il passaggio chiave di Machiavelli era questo ...,wagner,il passaggio chiave di machiavelli era questo ...,neutral,0.00,positive,0.949663
